# Xpense - Expense Tracker Application
## A comprehensive expense tracking application with pandas integration for enhanced data analysis

This notebook adapts the Xpense expense tracker to work in Jupyter environment with pandas for advanced data analysis capabilities.

In [ ]:
# Import Required Libraries
import tkinter as tk
import customtkinter as ctk
import mysql.connector
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from datetime import datetime
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# For Jupyter notebook display
from IPython.display import display, HTML
import threading
import sys

print("All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"CustomTkinter version: {ctk.__version__}")

In [ ]:
# Database Configuration and Connection
def setup_database_connection():
    """Set up MySQL database connection with error handling"""
    try:
        conn = mysql.connector.connect(
            database="budget_planning",
            user="root",
            host="localhost",
            password="",
            port=3307,
        )
        print("✅ Database connection established successfully!")
        return conn
    except mysql.connector.Error as err:
        print(f"❌ Database connection failed: {err}")
        print("Please ensure MySQL server is running and credentials are correct.")
        return None

# Initialize database connection
conn = setup_database_connection()

# Database setup function
def create_tables_if_not_exist():
    """Create necessary tables if they don't exist"""
    if not conn:
        print("No database connection available")
        return
    
    try:
        cur = conn.cursor()
        
        # Create userinfo table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS userinfo (
                userid VARCHAR(50) PRIMARY KEY,
                password VARCHAR(100) NOT NULL,
                user_name VARCHAR(100) NOT NULL
            )
        """)
        
        # Create expense table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS expense (
                id INT AUTO_INCREMENT PRIMARY KEY,
                userid VARCHAR(50) NOT NULL,
                date DATE NOT NULL,
                title VARCHAR(200) NOT NULL,
                expense_type VARCHAR(50) NOT NULL,
                amount DECIMAL(10,2) NOT NULL,
                comment TEXT,
                FOREIGN KEY (userid) REFERENCES userinfo(userid)
            )
        """)
        
        conn.commit()
        print("✅ Database tables verified/created successfully!")
        
    except Exception as e:
        print(f"❌ Error creating tables: {e}")

# Run table creation
if conn:
    create_tables_if_not_exist()

In [ ]:
# Data Models and Database Operations
class ExpenseDatabase:
    """Class to handle all database operations"""
    
    def __init__(self, connection):
        self.conn = connection
    
    def authenticate_user(self, userid, password):
        """Authenticate user login"""
        try:
            cur = self.conn.cursor()
            cur.execute("SELECT userid, user_name FROM userinfo WHERE userid = %s AND password = %s", 
                       (userid, password))
            return cur.fetchone()
        except Exception as e:
            print(f"Authentication error: {e}")
            return None
    
    def check_user_exists(self, userid):
        """Check if user exists in database"""
        try:
            cur = self.conn.cursor()
            cur.execute("SELECT userid FROM userinfo WHERE userid = %s", (userid,))
            return cur.fetchone() is not None
        except Exception as e:
            print(f"Error checking user: {e}")
            return False
    
    def create_user(self, userid, password, user_name):
        """Create new user account"""
        try:
            cur = self.conn.cursor()
            cur.execute("INSERT INTO userinfo (userid, password, user_name) VALUES (%s, %s, %s)",
                       (userid, password, user_name))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error creating user: {e}")
            return False
    
    def add_expense(self, userid, title, expense_type, amount, comment=""):
        """Add new expense record"""
        try:
            cur = self.conn.cursor()
            cur.execute(
                "INSERT INTO expense (userid, date, title, expense_type, amount, comment) VALUES (%s, %s, %s, %s, %s, %s)",
                (userid, datetime.now().strftime("%Y-%m-%d"), title, expense_type, amount, comment)
            )
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error adding expense: {e}")
            return False
    
    def get_user_expenses(self, userid, limit=None):
        """Get all expenses for a user"""
        try:
            cur = self.conn.cursor()
            query = "SELECT id, date, title, expense_type, amount, comment FROM expense WHERE userid = %s ORDER BY date DESC"
            if limit:
                query += f" LIMIT {limit}"
            cur.execute(query, (userid,))
            return cur.fetchall()
        except Exception as e:
            print(f"Error fetching expenses: {e}")
            return []
    
    def update_expense(self, expense_id, title, expense_type, amount, comment):
        """Update existing expense record"""
        try:
            cur = self.conn.cursor()
            cur.execute(
                "UPDATE expense SET title = %s, expense_type = %s, amount = %s, comment = %s WHERE id = %s",
                (title, expense_type, amount, comment, expense_id)
            )
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error updating expense: {e}")
            return False
    
    def delete_expense(self, expense_id):
        """Delete expense record"""
        try:
            cur = self.conn.cursor()
            cur.execute("DELETE FROM expense WHERE id = %s", (expense_id,))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error deleting expense: {e}")
            return False

# Initialize database handler
if conn:
    db_handler = ExpenseDatabase(conn)
    print("✅ Database handler initialized successfully!")
else:
    db_handler = None
    print("❌ Database handler not available")

In [ ]:
# Pandas Integration for Data Analysis
class ExpenseAnalytics:
    """Class for advanced expense analytics using pandas"""
    
    def __init__(self, db_handler):
        self.db_handler = db_handler
    
    def get_expenses_dataframe(self, userid):
        """Convert expense data to pandas DataFrame"""
        try:
            expenses = self.db_handler.get_user_expenses(userid)
            if not expenses:
                return pd.DataFrame()
            
            df = pd.DataFrame(expenses, columns=['id', 'date', 'title', 'expense_type', 'amount', 'comment'])
            df['date'] = pd.to_datetime(df['date'])
            df['amount'] = pd.to_numeric(df['amount'])
            return df
        except Exception as e:
            print(f"Error creating DataFrame: {e}")
            return pd.DataFrame()
    
    def get_financial_summary(self, userid):
        """Get comprehensive financial summary"""
        df = self.get_expenses_dataframe(userid)
        if df.empty:
            return {
                'total_income': 0,
                'total_expenses': 0,
                'balance': 0,
                'expense_by_category': pd.DataFrame(),
                'monthly_trends': pd.DataFrame()
            }
        
        # Separate income and expenses
        income_df = df[df['expense_type'].isin(['Income', 'Allowance'])]
        expense_df = df[~df['expense_type'].isin(['Income', 'Allowance'])]
        
        total_income = income_df['amount'].sum()
        total_expenses = expense_df['amount'].sum()
        balance = total_income - total_expenses
        
        # Expense by category
        expense_by_category = expense_df.groupby('expense_type')['amount'].sum().reset_index()
        expense_by_category = expense_by_category.sort_values('amount', ascending=False)
        
        # Monthly trends
        df['month'] = df['date'].dt.to_period('M')
        monthly_income = income_df.groupby('month')['amount'].sum()
        monthly_expenses = expense_df.groupby('month')['amount'].sum()
        
        monthly_trends = pd.DataFrame({
            'income': monthly_income,
            'expenses': monthly_expenses
        }).fillna(0)
        monthly_trends['balance'] = monthly_trends['income'] - monthly_trends['expenses']
        
        return {
            'total_income': float(total_income),
            'total_expenses': float(total_expenses),
            'balance': float(balance),
            'expense_by_category': expense_by_category,
            'monthly_trends': monthly_trends
        }
    
    def get_spending_insights(self, userid):
        """Get detailed spending insights"""
        df = self.get_expenses_dataframe(userid)
        if df.empty:
            return {}
        
        expense_df = df[~df['expense_type'].isin(['Income', 'Allowance'])]
        
        insights = {
            'top_spending_category': expense_df.groupby('expense_type')['amount'].sum().idxmax() if not expense_df.empty else None,
            'average_transaction': float(expense_df['amount'].mean()) if not expense_df.empty else 0,
            'largest_expense': float(expense_df['amount'].max()) if not expense_df.empty else 0,
            'total_transactions': len(expense_df),
            'spending_by_day': expense_df.groupby(expense_df['date'].dt.day_name())['amount'].sum()
        }
        
        return insights

# Initialize analytics handler
if db_handler:
    analytics = ExpenseAnalytics(db_handler)
    print("✅ Analytics handler initialized successfully!")
else:
    analytics = None
    print("❌ Analytics handler not available")

In [ ]:
# Core Application Functions
class ExpenseTracker:
    """Main application class"""
    
    def __init__(self):
        self.db_handler = db_handler
        self.analytics = analytics
        self.current_user = None
        self.app = None
        
        # Custom color palette
        self.PRIMARY_COLOR = "#39ace7"
        self.SECONDARY_COLOR = "#0784b5" 
        self.DARK_COLOR = "#414c50"
        self.DARKER_COLOR = "#2d383c"
        self.DARKEST_COLOR = "#192428"
    
    def initialize_app(self):
        """Initialize the main application window"""
        self.app = ctk.CTk()
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
        self.app.geometry("1000x700")
        self.app.title("Xpense - Expense Tracker")
        return self.app
    
    def clear_window(self):
        """Clear all widgets from the window"""
        if self.app:
            for widget in self.app.winfo_children():
                widget.destroy()
    
    def login_user(self, userid, password):
        """Authenticate and login user"""
        if not self.db_handler:
            return False, "Database not available"
        
        if not userid or not password:
            return False, "Please fill in all fields"
        
        user = self.db_handler.authenticate_user(userid, password)
        if user:
            self.current_user = userid
            return True, "Login successful"
        else:
            user_exists = self.db_handler.check_user_exists(userid)
            if not user_exists:
                return False, "User not found"
            else:
                return False, "Invalid password"
    
    def register_user(self, userid, password, user_name):
        """Register new user"""
        if not self.db_handler:
            return False, "Database not available"
        
        if not all([userid, password, user_name]):
            return False, "Please fill in all fields"
        
        if self.db_handler.check_user_exists(userid):
            return False, "Username already exists"
        
        if self.db_handler.create_user(userid, password, user_name):
            self.current_user = userid
            return True, "Account created successfully"
        else:
            return False, "Failed to create account"
    
    def get_user_summary(self):
        """Get financial summary for current user"""
        if not self.current_user or not self.analytics:
            return None
        
        return self.analytics.get_financial_summary(self.current_user)
    
    def get_user_insights(self):
        """Get spending insights for current user"""
        if not self.current_user or not self.analytics:
            return None
        
        return self.analytics.get_spending_insights(self.current_user)
    
    def get_expenses_df(self):
        """Get expenses as pandas DataFrame"""
        if not self.current_user or not self.analytics:
            return pd.DataFrame()
        
        return self.analytics.get_expenses_dataframe(self.current_user)

# Initialize expense tracker
expense_tracker = ExpenseTracker()
print("✅ Expense Tracker initialized successfully!")